# Dynamic Programming

In [1]:
import gym
import gym_gridworlds
import numpy

## Policy Evaluation

In [2]:
env = gym.make('Gridworld-v0')
env.action_space, env.observation_space

(Discrete(4), Discrete(15))

In [3]:
stochastic_policy = numpy.ones((env.observation_space.n, env.action_space.n))
stochastic_policy /= env.action_space.n
stochastic_policy

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

In [4]:
def policy_evaluation(policy, env, gamma=1.0, epsilon=1e-5):
    V = numpy.zeros(env.observation_space.n)
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = V[s]
            v_sum = 0
            for a in range(env.action_space.n):
                v_sum += policy[s, a] * (env.P[a, s] @ (env.R[a, s] + gamma * V))
            V[s] = v_sum
            delta = max(delta, numpy.abs(v - V[s]))
        if delta < epsilon:
            return V

V = policy_evaluation(stochastic_policy, env)
V

array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569])

## Policy Improvement

In [5]:
def policy_improvement(env, V, gamma=1.0):
    policy = numpy.zeros((env.observation_space.n, env.action_space.n))
    for s in range(env.observation_space.n):
        action_values = numpy.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            action_values[a] = env.P[a, s] @ (env.R[a, s] + gamma * V)
        a = numpy.argmax(action_values)
        policy[s, a] = 1.0
    return policy

policy = policy_improvement(env, V)
policy

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [6]:
policy_evaluation(policy, env)

array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.])

## Policy Iteration

In [7]:
def policy_iteration(env, gamma=1.0):
    policy = numpy.ones((env.observation_space.n, env.action_space.n))
    policy /= env.action_space.n
    while True:
        V = policy_evaluation(policy, env, gamma)
        policy_prime = policy_improvement(env, V, gamma)
        if (policy == policy_prime).all():
            return policy, V
        policy = policy_prime

optimal_policy, optimal_V = policy_iteration(env)
optimal_policy, optimal_V

(array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]]),
 array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
        -2., -1.]))

## Value Iteration

In [8]:
def value_iteration(env, gamma=1.0, epsilon=1e-5):
    V = numpy.zeros(env.observation_space.n)
    while True:
        v = numpy.copy(V)
        V = numpy.max(env.R + (gamma * env.P @ V), axis=0)
        delta = numpy.max(numpy.abs(v - V))
        if delta < epsilon:
            break
    return policy_improvement(env, V, gamma), V
    

value_iteration(env)

(array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]]),
 array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
        -2., -1.]))